In [1]:
!pip install -q --upgrade pip
!pip install jaxlib==0.4.4
!pip install -q --upgrade jax[cuda11_cudnn805] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q git+https://www.github.com/google/neural-tangents
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.8/112.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 68.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cpu
Looking in indexe

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK


In [3]:
import neural_tangents as nt
from neural_tangents import stax
from jax import numpy as np
from jax import random
import jax

from utils import *

In [ ]:
# define a toy 4x4 graph
# 0----3
# |    |
# 1----2


edge_list = np.array([[0,1],
                      [0,3],
                      [1,0],
                      [1,2],
                      [2,1],
                      [2,3],
                      [3,2],
                      [3,0]])

A = to_dense(np.transpose(edge_list), 4)
print(A)

node_features = np.array([[1,2,3,4,5,6,7],
                          [10,20,30,40,50,60,70],
                          [100,200,300,400,500,600,700],
                          [1000,2000,3000,4000,5000,6000,7000]])

node_features = node_features * 1.0

node_features


NameError: ignored

In [ ]:
 expected = A @ node_features
 expected

Simple example much like presented here: https://neural-tangents.readthedocs.io/en/latest/_autosummary/neural_tangents.stax.Aggregate.html#neural_tangents.stax.Aggregate

The problem with this is, that we can not run convolution layer on this input shape?! We need to reshape the node features such that they have a channel dimension

In [ ]:
# reshape
# the node feature must be a n x 1 dimensional array, with a channel dim

# add a batch dimension
x = np.expand_dims(node_features, 0)

pattern = np.expand_dims(A, 0)

print(pattern.shape)
print(x.shape)

In [ ]:
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Aggregate(aggregate_axis=1, batch_axis=0, channel_axis=2)
)

key = random.PRNGKey(0)
key, subkey = jax.random.split(key)
_, params = init_fn(subkey, x.shape)

In [ ]:
out = apply_fn(params, x, pattern=pattern)
out == expected

Now with a channel dimension

In [ ]:
# reshape
# the node feature must be a n x 1 dimensional array, with a channel dim

# add a batch dimension
x = np.expand_dims(node_features, 0)

# add a dim to make the node features a n x 1 dimensional array
# with a channel dim
x = np.expand_dims(x, 2)

pattern = np.expand_dims(A, 0)
pattern = np.expand_dims(pattern, 2)
pattern = np.expand_dims(pattern, 4)

print(pattern.shape)
print(x.shape)

In [ ]:
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3),
    stax.Conv(1, (1,1), parameterization="standard")
)

key = random.PRNGKey(0)
key, subkey = jax.random.split(key)
_, params = init_fn(subkey, x.shape)

In [ ]:
# set all weights to 1 for the convolution
params = ((), (np.ones(params[1][0].shape), None))

In [ ]:
out = apply_fn(params, x, pattern=pattern)
np.squeeze(out).shape

In [ ]:
# the convolution weights are all 1 => the output of the convlution is the sum of the output of the agg layer
out = apply_fn(params, x, pattern=pattern)
print(np.squeeze(out))
np.sum(expected, axis=1)

Test the kernel function

In [ ]:
layer_wide = 16
# define a grap convolution network
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Conv(layer_wide, (1,1), parameterization="ntk"), stax.Relu(),
    stax.Aggregate(aggregate_axis=(1, 2), batch_axis=0, channel_axis=3, implementation="DENSE"),
    stax.Conv(layer_wide, (1,1), parameterization="ntk"), stax.Relu(),
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="DENSE"),
    stax.Conv(layer_wide, (1,1), parameterization="ntk"), stax.Relu(),
    stax.GlobalSumPool(),
    stax.Dense(1),
)

key = random.PRNGKey(0)
key, subkey = jax.random.split(key)
_, params = init_fn(subkey, x.shape)

a = apply_fn(params, x, pattern=pattern)
k = kernel_fn(x, x, ('ntk', "nngp"), pattern=(pattern, pattern))

print(k.ntk.shape)
print(k.nngp.shape)
print(x.shape)
print(pattern.shape)

Now with spars pattern
The shape of the pattern is: batch x #edges x 1 x 2

In [ ]:
# define a toy 4x4 graph
# 0----3
# |    |
# 1----2


edge_list = np.array([[0,1],
                      [0,3],
                      [1,0],
                      [1,2],
                      [2,1],
                      [2,3],
                      [3,2],
                      [3,0]])

node_features = np.array([[1,2,3,4,5,6,7],
                          [10,20,30,40,50,60,70],
                          [100,200,300,400,500,600,700],
                          [1000,2000,3000,4000,5000,6000,7000]])

node_features = node_features * 1.0

A = np.array([[0,1,0,1],
              [1,0,1,0],
              [0,1,0,1],
              [1,0,1,0]])

expected = A @ node_features

print(expected)

node_features


[[1010. 2020. 3030. 4040. 5050. 6060. 7070.]
 [ 101.  202.  303.  404.  505.  606.  707.]
 [1010. 2020. 3030. 4040. 5050. 6060. 7070.]
 [ 101.  202.  303.  404.  505.  606.  707.]]


Array([[1.e+00, 2.e+00, 3.e+00, 4.e+00, 5.e+00, 6.e+00, 7.e+00],
       [1.e+01, 2.e+01, 3.e+01, 4.e+01, 5.e+01, 6.e+01, 7.e+01],
       [1.e+02, 2.e+02, 3.e+02, 4.e+02, 5.e+02, 6.e+02, 7.e+02],
       [1.e+03, 2.e+03, 3.e+03, 4.e+03, 5.e+03, 6.e+03, 7.e+03]],      dtype=float32, weak_type=True)

In [ ]:
# reshape
# the node feature must be a n x 1 dimensional array, with a channel dim

# add a batch dimension
x = np.expand_dims(node_features, 0)

# add a dim to make the node features a n x 1 dimensional array
# with a channel dim
x = np.expand_dims(x, 2)

pattern = np.expand_dims(edge_list, 0)
pattern = np.expand_dims(pattern, 2)

print(pattern.shape)
print(x.shape)

(1, 8, 1, 2)
(1, 4, 1, 7)


In [ ]:
# need to add a copy at axis 2 to make the argegation over axis (1,2) work
# need to aggreagte over axis (1,2) to keep the shape of the kernel stable in the aggregation layer
pattern = np.append(pattern, pattern, axis=2)
print(pattern.shape)

(1, 8, 2, 2)


In [ ]:
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE")
)

key = random.PRNGKey(0)
key, subkey = jax.random.split(key)
_, params = init_fn(subkey, x.shape)

In [ ]:
out = apply_fn(params, x, pattern=pattern)
print(out.shape)
np.squeeze(out) == expected

(1, 4, 1, 7)


Array([[ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True]], dtype=bool)

Add conv layer

In [ ]:
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Conv(1, (1,1), parameterization="standard"),
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
)

key = random.PRNGKey(0)
key, subkey = jax.random.split(key)
_, params = init_fn(subkey, x.shape)

In [ ]:
# set all weights to 1 for the convolution
params = [(np.ones(params[0][0].shape), None), (), ()]

In [ ]:
# the convolution weights are all 1 => the output of the convlution is the sum of the output of the agg layer
out = apply_fn(params, x, pattern=pattern)
print(np.squeeze(out))
np.sum(expected, axis=1)

[28280.  2828. 28280.  2828.]


Array([28280.,  2828., 28280.,  2828.], dtype=float32)

In [ ]:
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Conv(10, (1,1), parameterization="standard"),
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
    stax.GlobalSumPool(),
)

key = random.PRNGKey(0)
key, subkey = jax.random.split(key)
_, params = init_fn(subkey, x.shape)

# set all weights to 1 for the convolution
params = [(np.ones(params[0][0].shape), None), (), ()]

In [ ]:
# the convolution weights are all 1 => the output of the convlution is the sum of the output of the agg layer
out = apply_fn(params, x, pattern=pattern)
print(np.squeeze(out))
np.sum(expected, axis=1)

[62216. 62216. 62216. 62216. 62216. 62216. 62216. 62216. 62216. 62216.]


Array([28280.,  2828., 28280.,  2828.], dtype=float32)

In [ ]:
pattern

Array([[[[0, 1],
         [0, 1]],

        [[0, 3],
         [0, 3]],

        [[1, 0],
         [1, 0]],

        [[1, 2],
         [1, 2]],

        [[2, 1],
         [2, 1]],

        [[2, 3],
         [2, 3]],

        [[3, 2],
         [3, 2]],

        [[3, 0],
         [3, 0]]]], dtype=int32)

Test the kernel function:

In [ ]:
pattern = np.append(pattern, pattern, axis=2)

In [ ]:
pattern.shape

(1, 8, 2, 2)

In [ ]:
print(x.shape)
print(pattern.shape)

(1, 4, 1, 7)
(1, 8, 2, 2)


In [ ]:
x = np.swapaxes(x, 0, 1)
pattern = np.swapaxes(pattern, 0, 1)

print(x.shape)
print(pattern.shape)

(4, 1, 1, 7)
(8, 1, 2, 2)


In [ ]:
layer_wide = 16
# define a grap convolution network
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Conv(layer_wide, (1,1), parameterization="ntk"), stax.Relu(),
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
    stax.Conv(layer_wide, (1,1), parameterization="ntk"), stax.Relu(),
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
    stax.Conv(layer_wide, (1,1), parameterization="ntk"), stax.Relu(),
    # stax.GlobalSumPool(),
    # stax.Dense(1),
)


print(x.shape)
print(pattern.shape)

key = random.PRNGKey(0)
key, subkey = jax.random.split(key)
_, params = init_fn(subkey, x.shape)

a = apply_fn(params, x, pattern=pattern)
k = kernel_fn(x, x, ('ntk', "nngp"), pattern=(pattern, pattern))

print(k.ntk.shape)
print(k.nngp.shape)

(1, 4, 1, 7)
(1, 8, 2, 2)
(1, 1, 1, 1, 4, 4)
(1, 1, 1, 1, 4, 4)


In [ ]:
pattern.shape

(1, 8, 2, 2)

# spars edge lists two graphs example

In [203]:
# define a toy 4x4 graph
# 0----3
# |    |
# 1----2
# and 2x2 graph
# 0----1

edge_list_1 = np.array([[0,1],
                      [0,3],
                      [1,0],
                      [1,2],
                      [2,1],
                      [2,3],
                      [3,2],
                      [3,0]])

node_features_1 = np.array([[1,2],
                          [11,12],
                          [21,22],
                          [31,32]])

node_features_1 = node_features_1 * 1.0

edge_list_2 = np.array([[0,1],
                        [1,0],
                        [0,0],
                        [1,1]]) + 4

node_features_2 = np.array([[1,2],
                          [21,22]])

node_features_2 = node_features_2 * 1.0 

graph_indx = np.append(np.full(node_features_1.shape[0], 0), np.full(node_features_2.shape[0], 1), 0)
edge_list = np.append(edge_list_1, edge_list_2, 0)
node_features = np.append(node_features_1, node_features_2, 0)

print(graph_indx.shape)
print(edge_list.shape)
print(node_features.shape)

node_features = np.expand_dims(node_features, 0)
node_features = np.expand_dims(node_features, 2)

# need this to be able to aggregat over both axis, which we need for the kernel function to work
edge_list = np.expand_dims(edge_list, 0)
edge_list = np.expand_dims(edge_list, 2)
edge_list = np.append(edge_list, edge_list, axis=2)

print(edge_list.shape)
print(node_features.shape)

edge_list

(6,)
(12, 2)
(6, 2)
(1, 12, 2, 2)
(1, 6, 1, 2)


Array([[[[0, 1],
         [0, 1]],

        [[0, 3],
         [0, 3]],

        [[1, 0],
         [1, 0]],

        [[1, 2],
         [1, 2]],

        [[2, 1],
         [2, 1]],

        [[2, 3],
         [2, 3]],

        [[3, 2],
         [3, 2]],

        [[3, 0],
         [3, 0]],

        [[4, 5],
         [4, 5]],

        [[5, 4],
         [5, 4]],

        [[4, 4],
         [4, 4]],

        [[5, 5],
         [5, 5]]]], dtype=int32)

In [204]:
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Conv(5, (1,1), parameterization="standard"),
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
)

key = random.PRNGKey(0)
key, subkey = jax.random.split(key)
_, params = init_fn(subkey, node_features.shape)

# set all weights to 1 for the convolution
params = [(np.ones(params[0][0].shape), None), (), ()]

In [205]:
out = apply_fn(params, node_features, pattern=edge_list)
print(out.shape)
np.squeeze(out)

(1, 6, 1, 5)


Array([[86., 86., 86., 86., 86.],
       [46., 46., 46., 46., 46.],
       [86., 86., 86., 86., 86.],
       [46., 46., 46., 46., 46.],
       [46., 46., 46., 46., 46.],
       [46., 46., 46., 46., 46.]], dtype=float32)

In [180]:
edge_list.shape

(1, 12, 2)

In [208]:
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Conv(5, (1,1), parameterization="standard"),
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
    stax.Conv(5, (1,1), parameterization="standard"),
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
)


k = kernel_fn(node_features, node_features, ('ntk'), pattern=(edge_list, edge_list))
np.squeeze(k).shape

(6, 6)

In [274]:
a = np.array([1,2,3])
b = np.array([10,20,30])

np.transpose(np.array([a,b]))

Array([[ 1, 10],
       [ 2, 20],
       [ 3, 30]], dtype=int32)

In [271]:
from typing import Callable, Iterable, Optional, Sequence, Tuple, Union
from neural_tangents import Kernel
from neural_tangents._src.stax.requirements import (
    Bool,
    Diagonal,
    get_diagonal_outer_prods,
    layer,
    mean_and_var,
    requires,
    supports_masking,
)


@layer
@supports_masking(remask_kernel=False)
def index_aggregation():
  init_fn = lambda rng, input_shape: (input_shape, ())

  def apply_fn(
        params, inputs: np.ndarray, *, pattern: Optional[np.ndarray] = None,
        graph_indx: Optional[np.ndarray] = None, 
        nb_graphs: Optional[int] = None,
        **kwargs
    ):

    return np.apply_along_axis(lambda x: jax.ops.segment_sum(x, graph_indx, num_segments=nb_graphs), 0, np.squeeze(inputs))

  def kernel_fn(
        k: Kernel,
        *,
        pattern: Tuple[Optional[np.ndarray], Optional[np.ndarray]] = (None, None),
        graph_indx: Tuple[Optional[np.ndarray], Optional[np.ndarray]] = (None, None),
        nb_graphs: Tuple[Optional[int], Optional[int]] = (None, None),
        **kwargs
    ):

    def agg(x, kernel_graph_indx):
      agg_x = jax.ops.segment_sum(np.reshape(x, (-1)), kernel_graph_indx, num_segments=nb_graphs[0]*nb_graphs[1])
      agg_x = np.reshape(agg_x, nb_graphs)
      agg_x = np.expand_dims(agg_x, 0)
      agg_x = np.expand_dims(agg_x, 0)
      return agg_x

    k_prod_graph_indx = row_wise_karthesian_prod(np.expand_dims(graph_indx[0], 1), np.expand_dims(graph_indx[1], 1))
    kernel_graph_indx = np.ravel_multi_index([k_prod_graph_indx[:,0], k_prod_graph_indx[:,1]], nb_graphs)

    # Todo:
    # Do I need to calculate the cov1, cov2 too?

    agg_ntk = agg(k.ntk, kernel_graph_indx)
    agg_nngp = agg(k.nngp, kernel_graph_indx)
    
    return k.replace(ntk=agg_ntk, nngp=agg_nngp, is_gaussian=True, is_input=False, 
                     channel_axis=1)

  return init_fn, apply_fn, kernel_fn
  


## Test the apply function

In [218]:
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Conv(5, (1,1), parameterization="standard"),
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
    index_aggregation(),
    # stax.Dense(1)
)

key = random.PRNGKey(0)
key, subkey = jax.random.split(key)
_, params = init_fn(subkey, node_features.shape)
params

/usr/local/lib/python3.9/dist-packages/neural_tangents/_src/stax/linear.py:439: UserWarning: Negative indices in `pattern` are considered as padding (i.e. ignored), unlike typical numpy negative indexing.
  warnings.warn('Negative indices in `pattern` are considered as padding '


[(Array([[[[-1.2358007 ,  1.0902201 , -0.17007037, -0.12051605,
             0.8951071 ],
           [ 0.20728225, -0.6802286 ,  0.16402271,  0.60656357,
             0.7451222 ]]]], dtype=float32), None), (), ()]

In [219]:
# set all weights to 1 for the convolution
params[0] = (np.ones(params[0][0].shape), None)
params

[(Array([[[[1., 1., 1., 1., 1.],
           [1., 1., 1., 1., 1.]]]], dtype=float32), None), (), ()]

In [220]:
out = apply_fn(params, node_features, pattern=edge_list, graph_indx=graph_indx)
print(out.shape)
np.squeeze(out)

(2, 5)


Array([[264., 264., 264., 264., 264.],
       [ 92.,  92.,  92.,  92.,  92.]], dtype=float32)

## Test the kernel function

In [221]:
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Conv(5, (1,1), parameterization="standard"),
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
    # index_aggregation(),
    # stax.Dense(1)
)

k = kernel_fn(node_features, node_features, ('ntk'), pattern=(edge_list, edge_list), graph_indx=graph_indx)
np.squeeze(k).shape

(6, 6)

In [242]:
# this is the expected result of the kernel function of the index_aggregation
# the input has 2 graphs => the kernel is a 2x2 array
# the layer bevor the aggregation is 6x6 (because 6 nodes in input)
# the first 4 nodes are from graph one => the sum of the 4x4 upper left block matrix is the "self kernel" of graph 1
# the las 2 nodes are from graph two => the sum of the 2x2 lower right block matrix is the "self kernel" of graph 2
# the ramaining 2x4 and 4x2 block matricies are the kernel of graph 1 graph 2 and vice versa
kernel = np.squeeze(k)
print(np.sum(kernel[:4,:4]))
print(np.sum(kernel[-2:,-2:]))
print(np.sum(kernel[:4,-2:]))
print(np.sum(kernel[:-2,4:]))

34880.0
4240.0
12160.0
12160.0


In [268]:
# now with the aggregation
# results match the expectation
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Conv(5, (1,1), parameterization="standard"),
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
    index_aggregation(),
    # stax.Dense(1)
)

k = kernel_fn(node_features, node_features, ('ntk'), pattern=(edge_list, edge_list), graph_indx=(graph_indx, graph_indx), nb_graphs=(2,2))
k

Array([[[[34880., 12160.],
         [12160.,  4240.]]]], dtype=float32)

In [272]:
# now with the aggregation
# after the dense layer
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Conv(5, (1,1), parameterization="standard"),
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
    index_aggregation(),
    stax.Dense(1)
)

k = kernel_fn(node_features, node_features, ('ntk'), pattern=(edge_list, edge_list), graph_indx=(graph_indx, graph_indx), nb_graphs=(2,2))
k

Array([[[[52320., 18240.],
         [18240.,  6360.]]]], dtype=float32)

# Batches of spars edge lists

In [ ]:
# define a toy 4x4 graph
# 0----3
# |    |
# 1----2
# and 2x2 graph
# 0----1

edge_list_1 = np.array([[0,1],
                      [0,3],
                      [1,0],
                      [1,2],
                      [2,1],
                      [2,3],
                      [3,2],
                      [3,0]])

node_features_1 = np.array([[1,2,3,4,5,6,7],
                          [10,20,30,40,50,60,70],
                          [100,200,300,400,500,600,700],
                          [1000,2000,3000,4000,5000,6000,7000]])

node_features_1 = node_features_1 * 1.0

edge_list_2 = np.array([[0,1]])

node_features_2 = np.array([[1,2,3,4,5,6,7],
                          [10,20,30,40,50,60,70]])

node_features_2 = node_features_2 * 1.0


edge_list = np.expand_dims(edge_list_1, 0)
edge_list_2_zeros = np.zeros(edge_list_1.shape)
edge_list_2_zeros = edge_list_2_zeros.at[:edge_list_2.shape[0],:edge_list_2.shape[1]].set(edge_list_2)
edge_list_2_zeros = np.expand_dims(edge_list_2_zeros, 0)
edge_list = np.append(edge_list, edge_list_2_zeros, 0)


node_features = np.expand_dims(node_features_1, 0)
node_features_2_zeros = np.zeros(node_features_1.shape)
node_features_2_zeros = node_features_2_zeros.at[:node_features_2.shape[0],:node_features_2.shape[1]].set(node_features_2)
node_features_2_zeros = np.expand_dims(node_features_2_zeros, 0)
node_features = np.append(node_features, node_features_2_zeros, 0)
node_features = np.expand_dims(node_features, 2)

pattern = edge_list
pattern = np.expand_dims(pattern, -1)
pattern = np.append(pattern, pattern, axis=-1)
pattern = np.array(pattern, dtype="int32")
print(pattern.shape)
print(node_features.shape)

(2, 8, 2, 2)
(2, 4, 1, 7)


In [ ]:
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Conv(10, (1,1), parameterization="ntk"), stax.Relu(),
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
    # stax.GlobalSumPool(),
    # stax.Dense(1)
)

key = random.PRNGKey(0)
key, subkey = jax.random.split(key)
_, params = init_fn(subkey, x.shape)

In [ ]:
out = apply_fn(params, node_features, pattern=pattern)
print(out.shape)
np.squeeze(out)

(2, 4, 1, 10)


Array([[[0.0000000e+00, 2.9890306e+00, 5.7539976e-01, 1.0779002e+01,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 8.0347910e+00,
         1.8533856e+00, 1.0105654e+01],
        [0.0000000e+00, 2.9890301e+01, 5.7540030e+00, 1.0779002e+02,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 8.0347908e+01,
         1.8533855e+01, 1.0105654e+02],
        [0.0000000e+00, 2.9890305e+02, 5.7540031e+01, 1.0779003e+03,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 8.0347913e+02,
         1.8533858e+02, 1.0105653e+03],
        [0.0000000e+00, 2.9890312e+03, 5.7540045e+02, 1.0779003e+04,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 8.0347910e+03,
         1.8533855e+03, 1.0105653e+04]],

       [[0.0000000e+00, 1.1956122e+01, 2.3015993e+00, 4.3116005e+01,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 3.2139164e+01,
         7.4135427e+00, 4.0422611e+01],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00, 